In [35]:
import tensorflow
import keras.layers as layers
import keras.models as models
import keras
import numpy as np
ACTION_SPACE = [i for i in range(5)]
ROWS = 6
COLUMNS = 6
OUTPUT_SIZE = len(ACTION_SPACE)
INPUT_SHAPE = (ROWS * COLUMNS,)
DEVICE = "CPU"

In [2]:
tensorflow.debugging.set_log_device_placement(True)

In [3]:
with tensorflow.device("CPU"):
    tensorflow.random.normal((200, 3))

Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0


In [24]:
def make_zombie_model():
    """
    makes the model that will be used for zombies
    The output of the model will be the predicted q value
    for being in a certain state.
    """
    model = models.Sequential()
    model.add(layers.InputLayer(INPUT_SHAPE))
    model.add(layers.Flatten())
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(128))  # 120 is arbitrary number
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(OUTPUT_SIZE))
    return model

In [36]:
with tensorflow.device(DEVICE):
    model = make_zombie_model()

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localho

In [26]:
print(model.input_shape)
model.summary()

(None, 36, 1)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 36)                0         
                                                                 
 dense_9 (Dense)             (None, 64)                2368      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 5)                 645       
                                                                 
Total params: 11,333
Trainable params: 1

In [37]:
with tensorflow.device(DEVICE):
    temp = model(tensorflow.random.normal((1, 36)), training=False)
print(temp.shape)

Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BiasAdd in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LeakyRelu in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BiasAdd in device /job:localhost/replica

### Optimizers and Loss

In [38]:
with tensorflow.device(DEVICE):
    optimizer = keras.optimizers.Adam(0.004)
    loss = keras.losses.MeanSquaredError()

In [ ]:
tensorflow.Transit